In [1]:
#import sys
#import pandas as pd
#import seaborn as sns
#from time import time
#from pyspark.sql import Row
#from pyspark.sql import SparkSession
#import matplotlib.pyplot as plt
#from pyspark.ml.feature import StringIndexer
#from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit,CrossValidator
#from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
#from pyspark.mllib.recommendation import ALS

from pyspark.mllib.recommendation import MatrixFactorizationModel, Rating

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import Row


from pyspark.ml import Pipeline, PipelineModel
#from collections import namedtuple

In [59]:
# data = sc.textFile("hdfs://devenv/user/spark/mllib_als/ratings.dat")
# ratings = data.map(lambda l: l.split('::')).map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))
# ratings.take(5)

[Rating(user=1, product=122, rating=5.0),
 Rating(user=1, product=185, rating=5.0),
 Rating(user=1, product=231, rating=5.0),
 Rating(user=1, product=292, rating=5.0),
 Rating(user=1, product=316, rating=5.0)]

In [4]:
data = spark.read.text("hdfs://master/user/spark/mllib_ALS/ratings.dat").rdd
#data.take(5)
parts = data.map(lambda row: row.value.split("::"))
#parts.take(5)
#ratingsRDD.take(5)
ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), productId=int(p[1]), rating=float(p[2])))
ratings = spark.createDataFrame(ratingsRDD)
ratings.show()

+---------+------+------+
|productId|rating|userId|
+---------+------+------+
|      122|   5.0|     1|
|      185|   5.0|     1|
|      231|   5.0|     1|
|      292|   5.0|     1|
|      316|   5.0|     1|
|      329|   5.0|     1|
|      355|   5.0|     1|
|      356|   5.0|     1|
|      362|   5.0|     1|
|      364|   5.0|     1|
|      370|   5.0|     1|
|      377|   5.0|     1|
|      420|   5.0|     1|
|      466|   5.0|     1|
|      480|   5.0|     1|
|      520|   5.0|     1|
|      539|   5.0|     1|
|      586|   5.0|     1|
|      588|   5.0|     1|
|      589|   5.0|     1|
+---------+------+------+
only showing top 20 rows



In [5]:
(train, test) = ratings.randomSplit([0.8, 0.2], seed=1111)
#train.take(5)

In [6]:
als = ALS(rank=10, maxIter=5, regParam=0.01, userCol="userId", itemCol="productId", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(train)

In [8]:
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)
# Generate top 10 user recommendations for each movie
movieRecs = model.recommendForAllItems(10)

Root-mean-square error = 0.8185293794213098


In [17]:
# Save and load model
from pyspark.ml.recommendation import ALSModel
model.save("hdfs://master/user/spark/mllib_ALS/myCollaborativeFilter")
modelload=ALSModel.load("hdfs://master/user/spark/mllib_ALS/myCollaborativeFilter")

In [19]:
userRecs = modelload.recommendForAllUsers(10)
userRecs.show(100, truncate=False)

+------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|userId|recommendations                                                                                                                                                                                    |
+------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|148   |[[8458, 7.788329], [27436, 7.359067], [5518, 7.281373], [7343, 7.147215], [59418, 6.8846946], [6691, 6.8774347], [26147, 6.866016], [55417, 6.825425], [61253, 6.824204], [8506, 6.8109984]]       |
|463   |[[44653, 7.365165], [27736, 6.1425724], [5748, 6.042515], [63329, 6.030448], [30890, 5.9810886], [6691, 5.9219265], [6465, 5.692775], [8480, 5.6540365], [63121, 5.624399], 

In [21]:
movieRecs = modelload.recommendForAllItems(10)
movieRecs.show(100, truncate=False)

+---------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|productId|recommendations                                                                                                                                                                                         |
+---------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|148      |[[59073, 8.482101], [31695, 6.848071], [19365, 6.745085], [38976, 6.5568876], [66583, 6.548161], [4731, 6.4553027], [40413, 6.3316936], [32362, 6.310933], [28989, 6.290739], [25932, 6.278509]]        |
|463      |[[62350, 7.945484], [37512, 7.6501117], [5289, 7.2290163], [68796, 7.1860237], [52074, 7.1129613], [43835, 7.011845], [32235, 6.8522844],